# ホジキン・ハクスレー(Hodgkin-Huxley)モデル
ヤリイカの巨大軸索から電気生理記録によって解析を行い、Na$^+$イオンとK$^+$イオンの電流、とくに電位依存性コンダクタンスと呼ばれる仕組みが重要であることを示した

Na+チャネル、K+チャネルの開閉を実験的に測定し、開閉の非線形な動態を微分方程式を含む数式で表した

## 微分方程式

> $ C\frac{dV}{dt}= -\bar{g}_{leak}(V(t) - E_{leak}) - g_{Na}(V,t)(V(t) - E_{Na}) - g_{K}(V,t)(V(t) - E_{K})  + I_{ext}(t)$

- $C$は膜のキャパシタンス [μF/cm$^2$]

- $V(t)$は膜電位 [mV]

- $\bar{g}_{leak}$は定数、リークコンダクタンス[ mS/cm$^2$]

- $E_{leak}$は主にCl$^-$イオンの反転電位 [mV]

- $E_{Na}$は主にNa$^+$イオンの反転電位 [mV]

- $g_K(V,t)$は電位依存のK$^+$チャネルコンダクタンス [mS/cm$^2$]

- $E_K$はK$^+$イオンの反転電位 [mV]

- $I_{ext}(t)$は細胞の外部から注入する電流 [μA/cm$^2$]

キャパシタの総容量やコンダクタンス値を決定するチャネルの個数は膜の面積に比例するため、単位面積あたりに正規化されている



> $ g_{Na}(V,t) = \bar{g}_{Na}m^3(V,t)h(V,t) $

> $ g_{K}(V,t) = \bar{g}_K n^4(V,t) $

- $ g_{Na}(V,t) $, $ g_{K}(V,t)$ は最大コンダクタンス[mS/cm$^2$]の定数

- $m(V,t), h(V,t), n(V,t)$は膜電位に依存する**ゲート変数**

### ゲート変数

イオンチャネルは、単体では開状態と閉状態との2状態のみの素子。開状態と閉状態との間を確率的に遷移する。 閉状態から開状態への遷移確率が$\alpha$、開状態から閉状態への遷移確率が$\beta$である。

$$
\frac{dx}{dt}=\alpha_{x}(V)(1-x)-\beta_{x}(V)x
$$

$$
\begin{array}{ll}
\alpha_{m}(V)=\dfrac {0.1(25-V)}{\exp \left[(25-V)/10\right]-1}, &\beta_{m}(V)=4\exp {(-V/18)}\\
\alpha_{h}(V)=0.07\exp {(-V/20)}, & \beta_{h}(V)={\dfrac{1}{\exp {\left[(30-V)/10 \right]}+1}}\\
\alpha_{n}(V)={\dfrac {0.01(10-V)}{\exp {\left[(10-V)/10\right]}-1}},& \beta_{n}(V)=0.125\exp {(-V/80)} 
\end{array}
$$

## シミュレーション

In [ ]:
import math

E_REST = -65.0 # mV
C = 1.0 # μF/cm^2
G_LEAK = 0.3 # mS/cm^2
E_LEAK = 10.6 + E_REST # mV
G_Na = 120.0 # mS/cm^2
E_Na = 115.0 + E_REST
G_K = (  36.0 ) # mS / cm^2
E_K = ( -12.0 + ( E_REST ) ) # mV


DT = 0.01 # 10 micro s
T = 1000 # 1000 ms
NT =  100000  # T / DT

def alpha_m(v):
    return ( 2.5 - 0.1 * ( v - E_REST ) ) / ( math.exp( 2.5 - 0.1 * ( v - E_REST ) ) - 1.0 )

def beta_m(v):
    return 4.0 * math.exp( - ( v - E_REST ) / 18.0 )

def alpha_h(v):
    return 0.07 * math.exp ( - ( v - E_REST ) / 20.0 )

def beta_h(v):
    return 1.0 / (math.exp ( 3.0 - 0.1 * ( v - E_REST ) ) + 1.0 )

def alpha_n (v):
    return ( 0.1 - 0.01 * ( v - E_REST ) ) / ( math.exp ( 1 - 0.1 * ( v - E_REST ) ) - 1.0 )

def beta_n(v):
    return 0.125 * math.exp ( - ( v - E_REST ) / 80.0 )


def m0(v):
    return alpha_m (v) / ( alpha_m ( v ) + beta_m ( v ) )

def h0(v):
    return alpha_h ( v ) / ( alpha_h ( v ) + beta_h ( v ) )

def n0(v):
    return alpha_n ( v ) / ( alpha_n ( v ) + beta_n ( v ) )

def tau_m(v):
    return 1. / ( alpha_m ( v ) + beta_m ( v ) )

def tau_h(v):
    return 1. / ( alpha_h ( v ) + beta_h ( v ) )

def tau_n(v):
    return 1. / ( alpha_n ( v ) + beta_n ( v ) )


def dmdt (v, m):
    return ( 1.0 / tau_m ( v ) ) * ( - m + m0 ( v ) )

def dhdt(v, h):
    return ( 1.0 / tau_h ( v ) ) * ( - h + h0 ( v ) )

def dndt(v, n):
    return ( 1.0 / tau_n ( v ) ) * ( - n + n0 ( v ) )

def dvdt (v,  m, h, n, i_ext ):
    return ( - G_LEAK * ( v - E_LEAK ) - G_Na * m * m * m * h * ( v - E_Na ) - G_K * n * n * n * n * ( v - E_K ) + i_ext ) / C

v = E_REST
m = m0(v)
h = h0(v)
n = n0(v)

i_ext = 9.0 # microA / cm^2

for nt in range(NT):
    t = DT * nt
    #print(t, v, m, h, n)

    with open('hh.dat', 'a') as f:
        print(t, v, m, h, n, file=f)
    
    dmdt1 = dmdt ( v, m )
    dhdt1 = dhdt ( v, h )
    dndt1 = dndt ( v, n )
    dvdt1 = dvdt ( v, m, h, n, i_ext ); 

    dmdt2 = dmdt ( v + .5 * DT * dvdt1, m + .5 * DT * dmdt1 )
    dhdt2 = dhdt ( v + .5 * DT * dvdt1, h + .5 * DT * dhdt1 )
    dndt2 = dndt ( v + .5 * DT * dvdt1, n + .5 * DT * dndt1 )
    dvdt2 = dvdt ( v + .5 * DT * dvdt1, m + .5 * DT * dmdt1, h + .5 * DT * dhdt1, n + .5 * DT * dndt1, i_ext )

    dmdt3 = dmdt ( v + .5 * DT * dvdt2, m + .5 * DT * dmdt2 )
    dhdt3 = dhdt ( v + .5 * DT * dvdt2, h + .5 * DT * dhdt2 )
    dndt3 = dndt ( v + .5 * DT * dvdt2, n + .5 * DT * dndt2 )
    dvdt3 = dvdt ( v + .5 * DT * dvdt2, m + .5 * DT * dmdt2, h + .5 * DT * dhdt2, n + .5 * DT * dndt2, i_ext )

    dmdt4 = dmdt ( v + DT * dvdt3, m + DT * dmdt3 )
    dhdt4 = dhdt ( v + DT * dvdt3, h + DT * dhdt3 )
    dndt4 = dndt ( v + DT * dvdt3, n + DT * dndt3 )
    dvdt4 = dvdt ( v + DT * dvdt3, m + DT * dmdt3, h + DT * dhdt3, n + DT * dndt3, i_ext )

    m += DT * ( dmdt1 + 2 * dmdt2 + 2 * dmdt3 + dmdt4 ) / 6.
    h += DT * ( dhdt1 + 2 * dhdt2 + 2 * dhdt3 + dhdt4 ) / 6.
    n += DT * ( dndt1 + 2 * dndt2 + 2 * dndt3 + dndt4 ) / 6.
    v += DT * ( dvdt1 + 2 * dvdt2 + 2 * dvdt3 + dvdt4 ) / 6.

## シミュレーション結果

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.genfromtxt("hh.dat")

plt.plot(data[:,0], data[:,1])

plt.title("1000ms間の計算結果")
plt.xlabel("時間 [ms]")
plt.ylabel("膜電位 [mV]")

plt.show()
plt.savefig("hh 1000ms.png")